# Get More Data 
expand time horizon Jan 1 2019 - July 31, 2020 - 5 quarters. note 45 day (safe: 2 mo.) window from now (10/08/2020)

In [ ]:
# install.packages("readr")

In [ ]:
# https://stackoverflow.com/questions/63328348/invalid-value-at-start-index-type-uint64-1e05-invalid-issue-while-down
# remotes::install_github("r-dbi/bigrquery")

In [2]:
library(bigrquery)
library(plotly)
library(data.table)
library(stringr)
library(readr)

Loading required package: ggplot2


Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:stats’:

    filter


The following object is masked from ‘package:graphics’:

    layout




In [2]:
objects()

character(0)

In [3]:
projectid <- "radixbi-249015"

## count number of rows we're working with

In [4]:
expiry_date_start='2019-06-01'
expiry_date_end='2020-09-01'

source("../phaseII_06_fallbacksupp/query_string_defs_2.R")

# writeLines(count_expiry_data_str)

In [5]:
sql <- count_expiry_data_str

bq_table_download(bq_project_query(projectid, sql))

f0_
<int>
6876


In [ ]:
# result is wrong -- looking at something like 5.4M rows

In [ ]:
# webUI returns 5427633 rows -- WHY??

## 

In [7]:
projectid <- "radixbi-249015"
expiry_date_start='2019-06-01'
expiry_date_end='2020-09-01'

source("../phaseII_06_fallbacksupp/query_string_defs_2.R")

sql <- get_expiry_data_str

expiry_data <- bq_table_download(bq_project_query(projectid, sql))
print(dim(expiry_data))
print(object.size(expiry_data), units = "auto") 

# [1] 1743356      30
# 475.9 Mb

[1] 6876   28
2 Mb


In [ ]:
# saveRDS(expiry_data,"../../data/output/expiry_data")
# system("gsutil cp /home/jupyter/local/Domains_202003/data/output/* gs://data_outputt/output/")


## saved results of new query to bigquery table in project radixbi_2020


In [3]:
sql <- paste("SELECT COUNT(*) FROM `radix2020.expiry.expiry_20200902_20201102_20201127`")

bq_table_download(bq_project_query("radix2020", sql))

f0_
<int>
943779


In [6]:
sql <- paste("SELECT COUNT(*) FROM `radix2020.expiry.expiry_20190601_20200901_20201116`")

bq_table_download(bq_project_query("radix2020", sql))

f0_
<int>
5427638


In [ ]:
# result is correct -- right number of rows

# New data 11/27

In [5]:
# https://community.exploratory.io/t/google-bigquery-import-fails-with-invalid-value-at-start-index-type-uint64-1e-05-invalid/1901
options(scipen = 20)

In [6]:
sql <- paste("SELECT * FROM `radix2020.expiry.expiry_20200902_20201102_20201127`")

expiry_20200902_20201102_20201127 <- bq_table_download(bq_project_query("radix2020", sql))

In [7]:
dim(expiry_20200902_20201102_20201127)

[1] 943779     30

In [8]:
saveRDS(expiry_20200902_20201102_20201127,"../../data/output/datapull_20201127/expiry_20200902_20201102_20201127")
system("gsutil cp /home/jupyter/local/Domains_202003/data/output/datapull_20201127/* gs://data_outputt/output/")

## PROTOTYPE load data script

In [5]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [3]:
# load data
cat("Loading data...")
expiry_df <- readRDS("/home/jupyter/local/Domains_202003/data/output/expiry_20190601_20200901")

Loading data...

In [8]:
# select most recent 5Q [1 quarter = 90 days, 5 quarters = 450 days ]
# 450 days before 20200901 is 20190609 ... round off to 20190601
cat("Removing", expiry_df %>%filter(expiry_date < as.Date("2019-06-01") | expiry_date > as.Date("2020-09-01")) %>% tally() %>% pull(n) ,"rows due to expiry_date constraints")
expiry_df <- expiry_df %>% filter(expiry_date >= as.Date("2019-06-01") & expiry_date <= as.Date("2020-09-01"))

# remove renewed_count>1
cat("Removing", expiry_df %>% filter(renewed_count==1) %>% tally() %>% pull(n) ,"rows due to renewed_count constraints")
expiry_df <- expiry_df %>% filter(renewed_count==1)

# remove where gibb_score, etc. are NA
cat("Removing", expiry_df %>% filter(is.na(gibb_score)) %>% tally() %>% pull(n) ,"rows due to missing gibb_score")
expiry_df <- expiry_df %>% filter(!is.na(gibb_score))
cat("... now dataset min(creation_date) is ", expiry_df %>% summarise(min(creation_date)) %>% pull(1) %>% as.character(),".")

Removing 0 rows due to expiry_date constraintsRemoving 4661731 rows due to renewed_count constraintsRemoving 0 rows due to missing gibb_score... now dataset min(creation_date) is  2017-01-01 .

In [34]:
# remove where gibb_score, etc. are NA
cat("Removing", expiry_df %>% filter(tld=="pw" | tld=="in.net") %>% tally() %>% pull(n) ,"rows due to tld pw and in.net")
expiry_df <- expiry_df %>% filter(tld!="pw" & tld!="in.net") 
dim(expiry_df)

Removing 410868 rows due to tld pw and in.net

[1] 4250863      29

In [23]:
# exclude resellers  .pw and .in.net 
expiry_df %>% distinct(tld) %>% arrange(tld) %>% pull(tld)

[1] "fun"     "host"    "in.net"  "online"  "press"   "pw"      "site"   
 [8] "space"   "store"   "tech"    "website"

In [7]:
dim(expiry_df)

[1] 4661731      29

renewal_type,renewed_count,expiry_date,creation_date,status,domain_id,domain,registrar,reseller,reseller_country,⋯,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,sld_length,sld_type,sld_type2,tld
<chr>,<int>,<date>,<date>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<date>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>
FirstTime,1,2020-08-09,2019-08-09,Active,116566566,labong.in.net,PDR Ltd,Whois,United States,⋯,2020-08-09,6,8.66,labong,1,2,6,l,6l,in.net
FirstTime,1,2020-02-22,2019-02-22,Active,94523324,ygbh6vjkc3fuir0o.in.net,GMO,MuuMuu,Japan,⋯,2020-01-23,6,0.18,ygbh6vjkc3fuir0o,1,28,16,ln,16ln,in.net
FirstTime,1,2019-08-08,2018-08-08,Active,73838944,ldp4qse3twvcg6h.in.net,GMO,MuuMuu,Japan,⋯,2019-07-09,6,0.22,ldp4qse3twvcg6h,1,40,15,ln,15ln,in.net
FirstTime,1,2020-06-20,2019-06-20,Active,109112786,empowered.in.net,Namecheap,Namecheap,United States,⋯,2020-06-20,6,8.84,empowered,1,1,9,l,9l,in.net
FirstTime,1,2020-01-20,2019-01-20,Active,91482063,astounding.in.net,Namecheap,Namecheap,United States,⋯,2020-01-20,6,12.48,astounding,1,2,10,l,10l,in.net
FirstTime,1,2019-08-05,2018-08-05,Active,73623584,cheapuggsforsale.in.net,Namecheap,Namecheap,United States,⋯,2019-08-04,6,4.86,cheapuggsforsale,1,8,16,l,16l,in.net


In [28]:
expiry_df %>% filter(reseller=="dotserve inc") %>% head()


Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”
Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”


renewal_type,renewed_count,expiry_date,creation_date,status,domain_id,domain,registrar,reseller,reseller_country,⋯,renew_date,renew_arpt,gibb_score,pattern,pattern_domain_count,day_domains,sld_length,sld_type,sld_type2,tld
<chr>,<int>,<date>,<date>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,⋯,<date>,<dbl>,<dbl>,<chr>,<int>,<int>,<int>,<chr>,<chr>,<chr>


In [31]:
expiry_df %>% filter(tld=="pw") %>% tally() %>% pull(n) 
expiry_df %>% filter(tld=="in.net") %>% tally() %>% pull(n) 


[1] 396505

[1] 14363

In [32]:
dim(expiry_df)
expiry_df %>% filter(tld=="pw" | tld=="in.net") %>% tally() %>% pull(n) 
expiry_df %>% filter(tld!="pw" & tld!="in.net") %>% tally() %>% pull(n) 

[1] 4661731      29

[1] 410868

[1] 4250863

In [33]:
4250863+410868

[1] 4661731